In [ ]:
%pylab

In [ ]:
import control,timeit,time
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import spline
from timeit import default_timer as timer

In [ ]:
P = 1.8
I = 0.0
D = 0.000002

pid = PID2(P,I,D)
pid.SetPoint=0.0
pid.setSlewLimit(10.0)

END = 80
feedback = 0
debug = 1

feedback_list = []
time_list = []
setpoint_list = []

for i in range(1, END):
    if i>9:
        pid.SetPoint = 1

    pid.update(feedback)
    output = pid.output
    if pid.SetPoint > 0:
        feedback += (output - (1/i))
    time.sleep(0.02)

    feedback_list.append(feedback)
    setpoint_list.append(pid.SetPoint)
    time_list.append(i)

time_sm = np.array(time_list)
time_smooth = np.linspace(time_sm.min(), time_sm.max(), 300)
#feedback_smooth = spline(time_list, feedback_list, time_smooth)

plt.figure(1)
plt.plot(time_list, feedback_list)
plt.plot(time_list, setpoint_list)
plt.xlim((0, L))
#plt.ylim((min(feedback_list)-0.5, max(feedback_list)+0.5))
#plt.ylim((0,3))
plt.xlabel('time (s)')
plt.ylabel('PID (PV)')
plt.title('TEST PID')

#plt.ylim((1-0.5, 1+0.5))
plt.ylim((0,3))

plt.grid(True)
plt.show()

In [ ]:
class PID2:
    def __init__(self, P=0.2, I=0.0, D=0.0):
        self.Kp = P
        self.Ki = I
        self.Kd = D

        self.current_time = timer()
        self.last_time = self.current_time

        self.clear()

    def clear(self):
        """Clears PID computations and coefficients"""
        self.SetPoint = 0.0
        self.PTerm = 0.0
        self.ITerm = 0.0
        self.DTerm = 0.0
        self.last_error = 0.0
        self.last_output = 0.0
        self.int_error = 0.0
        self.windup_guard = 20.0
        self.slewrate = 0.0
        self.output = 0.0

    def update(self, feedback_value):
        """Calculates PID value for given reference feedback
        .. math::
            u(t) = K_p e(t) + K_i \int_{0}^{t} e(t)dt + K_d {de}/{dt}
        .. figure:: images/pid_1.png
           :align:   center
           Test PID with Kp=1.2, Ki=1, Kd=0.001 (test_pid.py)
        """
        error = self.SetPoint - feedback_value

        self.current_time = timer()
        delta_time = self.current_time - self.last_time
        delta_error = error - self.last_error

        self.PTerm = self.Kp * error
        self.ITerm += error * delta_time

        if (self.ITerm < -self.windup_guard):
            self.ITerm = -self.windup_guard
        elif (self.ITerm > self.windup_guard):
            self.ITerm = self.windup_guard

        self.DTerm = 0.0
        if delta_time > 0:
            self.DTerm = delta_error / delta_time      
        
        output_raw = self.PTerm + (self.Ki * self.ITerm) + (self.Kd * self.DTerm)
        if (self.slewrate > 0.0) :
            slew_scaled = min(self.slewrate*delta_time, self.slewrate) # so slew rate can only be reduced
            if (output_raw - self.last_output > slew_scaled):
                if (debug): print("Slew rate up limited to {:.3f} from {:.3f}".format(slew_scaled,output_raw - self.last_output))
                self.output = self.last_output + slew_scaled
            elif (output_raw - self.last_output < -slew_scaled):
                if (debug): print("Slew rate down limited to {:.3f} from {:.3f}".format(-slew_scaled,output_raw - self.last_output))
                self.output = self.last_output - slew_scaled
            else:
                if (debug): print("No slew rate limit - {}".format(output_raw))
                self.output = output_raw  
        else:
            if (debug): print("Slew limiter off - {}".format(output_raw))
            self.output = output_raw
            
        if (debug): print("P:{:.3f}|I:{:.3f}|D:{:.3f}|sp:{:.2f}".format(self.PTerm,self.Ki * self.ITerm,self.Kd * self.DTerm,self.SetPoint))
        
        # Remember things for next calculation
        self.last_output = self.output
        self.last_time = self.current_time
        self.last_error = error

    def setKp(self, proportional_gain):
        """Determines how aggressively the PID reacts to the current error with setting Proportional Gain"""
        self.Kp = proportional_gain

    def setKi(self, integral_gain):
        """Determines how aggressively the PID reacts to the current error with setting Integral Gain"""
        self.Ki = integral_gain

    def setKd(self, derivative_gain):
        """Determines how aggressively the PID reacts to the current error with setting Derivative Gain"""
        self.Kd = derivative_gain

    def setWindup(self, windup):
        """Integral windup, also known as integrator windup or reset windup,
        refers to the situation in a PID feedback controller where
        a large change in setpoint occurs (say a positive change)
        and the integral terms accumulates a significant error
        during the rise (windup), thus overshooting and continuing
        to increase as this accumulated error is unwound
        (offset by errors in the other direction).
        The specific problem is the excess overshooting.
        """
        self.windup_guard = windup
        
    def setSlewLimit(self, slew):
        """Slew rate limits serves to limit the output change to the rate specified per second
        """
        self.slewrate = slew
        
        